In [1]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression


url="https://raw.githubusercontent.com/ASIF-Mahmud1/Exploration/MLlessons/Miscellaneous/emailDataset.csv"
from io import StringIO
import string
import pandas as pd
import requests
s=requests.get(url).text

message_data=pd.read_csv(StringIO(s))
message_data.head()


,emailId,tag,mail-message
0,1,superior,"Dear Tom, I can't thank you enough for forward..."
1,2,casual,"Hi Linda, How's it going? Sorry I haven't been..."
2,3,casual,"Hey, Susi, whats wrong with you? Why aren't yo..."
3,4,superior,"Dear Barylior, I will need your opinion on how..."
4,5,superior,"Dear Liz,I really appreciate your understandin..."


In [2]:
message_data.groupby('tag').describe()


emailId                                                     
                  count       mean       std   min    25%   50%    75%   max
tag                                                                         
First Time          4.0  29.500000  1.290994  28.0  28.75  29.5  30.25  31.0
Regular meeting     2.0  22.500000  0.707107  22.0  22.25  22.5  22.75  23.0
Urgent meeting      5.0  24.600000  2.302173  21.0  24.00  25.0  26.00  27.0
casual             10.0  12.000000  6.324555   2.0   8.50  12.5  17.25  20.0
regular meeting     3.0  34.000000  1.000000  33.0  33.50  34.0  34.50  35.0
superior           11.0  11.090909  8.607608   1.0   5.50   9.0  15.00  32.0

In [ ]:
message_data_copy = message_data['mail-message'].copy() 

message_data_copy

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [5]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [6]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [7]:
message_data_copy = message_data_copy.apply(text_preprocess)
message_data_copy = message_data_copy.apply(stemmer)    # New Line
# message_data_copy

In [8]:
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_data_copy)


c:\users\kids\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass input=english as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [9]:
# message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
#                                                         message_data['tag'], test_size=0.3, random_state=20)

message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['tag'], random_state=20)

In [10]:
Spam_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
print("The prediction is :", pred)

The prediction is : ['superior' 'superior' 'superior' 'casual' 'superior' 'casual' 'casual'
 'superior' 'casual']


In [11]:
dummyEmail = ["Hey, Susi, whats wrong with you? Why aren't you "]
dummyEmailMatrix= vectorizer.transform(dummyEmail) 
dummyEmailMatrix
predict= Spam_model.predict(dummyEmailMatrix)
print("The prediction is :", predict)

The prediction is : ['casual']


In [12]:
prob_Of_Each_Class = Spam_model.predict_proba(dummyEmailMatrix)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % prob_Of_Each_Class[0])

Predicted Probabilities: [0.0901089  0.03343178 0.11895441 0.44645918 0.09298821 0.21805751]
